In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext


In [2]:
credentials_location = "C:/tools/gsutil/google_credentials.json"

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

 

In [3]:

sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [5]:
df_green = spark.read.parquet('gs://dtc_data_lake_de-zoomcamp-nytaxi-0001/pq/green/*/*')

In [7]:
df_green.count()

2304517

In [ ]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [ ]:
df_green.columns

In [ ]:
df_yellow.columns

In [ ]:
#Get columns that both datasets has
set(df_green.columns) & set (df_yellow.columns)

In [ ]:
df_green = df_green\
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime')\
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [ ]:
df_green.columns

In [ ]:
df_yellow = df_yellow\
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')\
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [ ]:
df_yellow.columns

In [ ]:
#Get columns that both datasets has
set(df_green.columns) & set (df_yellow.columns)

In [ ]:
common_cols = []

yellow_cols = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_cols:
        common_cols.append(col)

In [ ]:
common_cols

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df_green_sel = df_green\
    .select(common_cols)\
    .withColumn('service_type',F.lit('green'))

In [ ]:
df_yellow_sel = df_yellow\
    .select(common_cols)\
    .withColumn('service_type',F.lit('yellow'))

In [ ]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [ ]:
df_trips_data.groupBy('service_type').count().show()

In [ ]:
df_trips_data.registerTempTable('trips_data')

In [ ]:
spark.sql("""
SELECT * FROM trips_data LIMIT 10;
""").show()

In [ ]:
spark.sql("""
SELECT 
    service_type,
    count (1)
FROM 
    trips_data 
GROUP BY
    service_type
""").show()

In [ ]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [ ]:
df_result.show()

In [ ]:
df_result.registerTempTable('df_result_data')

In [ ]:
spark.sql("""
SELECT 
    service_type,
    count (1)
FROM 
    df_result_data 
GROUP BY
    service_type
""").show()